# Flair（PyTorch构建的NLP开发包）

## 安装

Flair安装需要Python 3.6

直接通过```pip install flair```即可

<img src='./image/flairinstall.png' />

## 使用训练好的预置分类模型

最新的Flair 0.4版本包含有两个预先训练好的模型。一个基于IMDB数据集训练的情感分析模型和一个攻击性语言探测模型（当前仅支持德语）。

只需一个命令就可以下载、存储并使用模型，这使得预置模型的使用过程异常简单。例如，下面的代码将使用情感分析模型：

当第一次运行以下代码时，Flair将下载情感分析模型，默认情况下会保存到本地用户主目录的.flair子目录，下载可能需要几分钟。（<b>作者太皮了，我下载到本地预估至少用了3个小时！！</b>）

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
print('load en-sentiment begin')
classifier = TextClassifier.load('en-sentiment')
print('load en-sentiment end')

sentence = Sentence('Flair is pretty neat!')
print('load predict begin')
classifier.predict(sentence)
print('load predict end')

# print sentence with predicted labels
print('Sentence above is: ', sentence.labels)

上面的代码首先载入必要的库，然后载入情感分析模型到内存中（必要时先下载），接下来就可以预测“Flair is pretty neat!”的情感分值了（0~1之间）。最后的命令输入结果为：

The sentence above is: [Positive (1.0)].

就是这么简单！现在你可以将上述代码整合为一个REST API，提供类似于google云端情感分析API的功能了！

## 使用训练好的预置分类模型

要训练一个自定义的文本分类器，首先需要一个标注文本集。Flair的分类数据集格式基于Facebook的FastText格式，要求在每一行的开始使用```**label**```前缀定义一个或多个标签。格式如下：

>```__label__<class_1> <text>
__label__<class_2> <text>```

在这篇文章中我们将使用Kaggle的SMS垃圾信息检测数据集来用Flair构建一个垃圾/非垃圾分类器。这个数据集很适合我们的学习任务，因为它很小，只有5572行数据，可以在单个CPU上只花几分钟就完成模型的训练。

## 预处理 - 构建数据集

首先下载Kaggle上的[数据集](https://www.kaggle.com/uciml/sms-spam-collection-dataset)，得到spam.csv；然后再数据集目录下，运行我们的处理脚本，得到训练集、开发集和测试集：

In [4]:
import pandas as pd
# The frac keyword argument specifies the fraction of rows to return in the random sample, 
# so frac=1 means return all rows (in random order).
data = pd.read_csv("./csv/spam.csv", encoding='latin-1').sample(frac=1).drop_duplicates()

data = data[['v1', 'v2']].rename(columns={"v1":"label", "v2":"text"})
 
data['label'] = '__label__' + data['label'].astype(str)

data.iloc[0:int(len(data)*0.8)].to_csv('./csv/spamtrain.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('./csv/spamtest.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('./csv/spamdev.csv', sep='\t', index = False, header = False)

上面的脚本会进行剔重和随机乱序处理，并按照80/10/10的比例进行数据集的分割。

脚本成功执行后，就会得到FastText格式的三个数据文件：train.csv、dev.csv和test.csv。

## 训练自定义文本分类模型

In [3]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
import os

if not os.path.exists('./nlpmodel/flair/spam/best-model.pt'):
    corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./csv'), 
                                                           test_file='spamtrain.csv', 
                                                           dev_file='spamdev.csv', 
                                                           train_file='spamtest.csv')

    word_embeddings = [WordEmbeddings('glove'), 
                       FlairEmbeddings('news-forward-fast'), 
                       FlairEmbeddings('news-backward-fast')]

    document_embeddings = DocumentLSTMEmbeddings(word_embeddings, 
                                                 hidden_size=512, 
                                                 reproject_words=True, 
                                                 reproject_words_dimension=256)

    classifier = TextClassifier(document_embeddings, 
                                label_dictionary=corpus.make_label_dictionary(), 
                                multi_label=False)

    trainer = ModelTrainer(classifier, corpus)

    trainer.train('./nlpmodel/flair/spam', max_epochs=20)
else:
    print('model: {0} existed!'.format('./nlpmodel/flair/spam/best-model.pt'))

model: ./nlpmodel/flair/spam/best-model.pt existed!


第一次运行上面这个脚本时，Flair会自动下载所需要的嵌入模型，这可能需要一段时间，然后接下来的整个训练过程还需要大约5分钟。

脚本首先载入需要的库和数据集，得到一个corpus对象。

接下来，我们创建一个嵌入列表，包含两个Flair上下文字符串嵌入和一个GloVe单词嵌入，这个列表接下来将作为我们文档嵌入对象的输入。

堆叠和文本嵌入是Flair中最有趣的感念之一，它们提供了将不同的嵌入整合在一起的手段，你可以同时使用传统的单词嵌入（例如GloVe、word2vector、ELMo）和Flair的上下文字符串嵌入。

在上面的示例中我们使用一个基于LSTM的方法来生成文档嵌入，关于该方法的详细描述可以参考[这里](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_5_DOCUMENT_EMBEDDINGS.md)。


下面进行简单的垃圾信息预测：

In [6]:
classifier = TextClassifier.load_from_file('./nlpmodel/flair/spam/best-model.pt')
textlist= ["Hi. Yes mum, I will...", 
           "Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur mob? Join the UK's largest Dogging Network bt Txting GRAVEL to 69888! Nt. ec2a. 31p.msg@150p"
          ]
for text in textlist:
    sentence = Sentence(text)
    classifier.predict(sentence)
    print('Text: {0} is belong to: {1}'.format(text, sentence.labels))


Text: Hi. Yes mum, I will... is belong to: [ham (1.0)]
Text: Want 2 get laid tonight? Want real Dogging locations sent direct 2 ur mob? Join the UK's largest Dogging Network bt Txting GRAVEL to 69888! Nt. ec2a. 31p.msg@150p is belong to: [spam (1.0)]


值得一提的是，经过加载预处理的GloVe单词嵌入生成的模型非常大，即使是Spam这种只有几千句，492K的训练集，生成的模型，也有200多M！

<img src='./image/flairmodel.png' />